In [ ]:
import numpy as np 
import math
import os
from tqdm.notebook import tqdm
from scipy.stats import ranksums
from statsmodels.stats.multitest import multipletests
from scipy.stats.mstats import kruskalwallis
import matplotlib as mlt
import matplotlib.lines as mlines
import glob
import matplotlib.pyplot as plt
import seaborn as sns

from numpy.random import randn
from numpy.random import seed
from numpy import mean
from numpy import var

from matplotlib import cm
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms


In [ ]:
def compute_pvalue(data1, data2, freq_vals): 
    
    pvalue = np.zeros(len(freq_vals))
    
    for iF in range(len(freq_vals)):
        _, pvalue[iF] = ranksums(data1[:,iF], data2[:,iF])
    
    freq_uncorrected = freq_vals[pvalue < 0.05]
        
    _, pvals_corrected, _, _ = multipletests(pvalue, method='fdr_bh')
    
    freq_corrected = freq_vals[pvals_corrected < 0.05]
        
    return freq_uncorrected, freq_corrected, pvalue, pvals_corrected
    
def compute_bootstraped_percentile(data, nTrials):
    nSubj, nFreqs = data.shape
    bootMat = np.zeros((nFreqs, nTrials))
    for i_trials in range(nTrials):
        idx_subj = np.random.randint(0,nSubj,nSubj)
        bootMat[:,i_trials] = np.nanmean(data[idx_subj,:], axis=0)
    return np.percentile(bootMat, (2.5,97.5), axis=1)

def plot_curve(ax, freq_vals, data, color, ls, label, a_size=6, alpha=1):
            
    def compute_bootstraped_percentile(data, nTrials):
        nSubj, nFreqs = data.shape
        bootMat = np.zeros((nFreqs, nTrials))
        for i_trials in range(nTrials):
            idx_subj = np.random.randint(0,nSubj,nSubj)
            bootMat[:,i_trials] = np.nanmean(data[idx_subj,:], axis=0)
        return np.percentile(bootMat, (2.5,97.5), axis=1)
    
    perc = compute_bootstraped_percentile(data,1000)
    
    ax.fill_between(freq_vals, perc[0,...], perc[1,...], alpha=0.1, color=color)
    
    ax.semilogx(freq_vals, data.mean(axis=0), color=color, label=label, linestyle=ls)

    ax.tick_params(labelsize=a_size)

def plot_stats(ax, freq, freq_corr, pos, color='black', marker='o', m_size=6):
    for xc in freq:
        ax.scatter(xc, pos, s=m_size, marker=marker, edgecolors=color, facecolors='none')
    
    for xc in freq_corr:
        ax.scatter(xc, pos, s=m_size, marker=marker,  edgecolors=color, facecolors=color)
        
def cohend(d1, d2, type='rm'):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)
    # calculate the pooled standard deviation
    if type=='rm':
        difference = d1 - d2
        s = np.std(difference)
    else:
        s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
            
    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

def plot_cohen(ax, freq, stats, pos, color='black'):

    for idxd, d in enumerate(stats):
        
        if d > 0.8: # large effect
            
            ax.scatter(freq[idxd], pos, marker='s', edgecolors=color, facecolors='none')
            
        if d > 0.5: # medium effect
            
            ax.scatter(freq[idxd], pos, marker='s', edgecolors=color, facecolors='none')

In [ ]:
def create_frequency_axis(f_min=2, f_max= 4, num_wavelet = 10):

    f_min_log = math.floor(np.log10(f_min))
    f_max_log = math.ceil(np.log10(f_max))
    mb = np.logspace(f_min_log, f_max_log, num=num_wavelet) # Morlet bank 
    freq_vals = mb[(2.1 < mb) & (mb < 90)] # Frequency of interest
    
    freq_labels = [('%.2f'%x) for x in freq_vals]
    
    return freq_vals, freq_labels
    
def extract_fei_new(files):
    
    name_subjs, data_masked = list(), list()
    for _, file in enumerate(tqdm(files)):
        
        name_subjs.append(file.split('/')[9][:6])
        
        tmp = np.load(file, allow_pickle=True)

        data_masked.append(np.nanmean(tmp[1,...],axis=0)) # tmp[1] == rimuovo fEI dove DFA < 0.6
    
    fEI_masked = np.array(data_masked)

    return name_subjs, fEI_masked

def extract_bis_new(files):
    
    data, name_subjs = list(), list()
    for ifile, file in enumerate(tqdm(files)):
        
        name_subjs.append(file.split('/')[9][:6])
        
        tmp = np.load(file, allow_pickle=True)
        
        data.append(np.mean(tmp,axis=0)) # Average channels
    
    bis = np.array(data)
    return name_subjs, bis

def extract_dfa_new(files):
    
    dfa, name_subjs = list(), list()
    for ifile, file in enumerate(tqdm(files)):
      
        name_subjs.append(file.split('/')[9][:6])
        datafile = np.load(file, allow_pickle=True)
        data = datafile.item()
        data_dfa = data['DFA']
        dfa.append(np.nanmean(data_dfa,axis=0))
    
    dfa = np.array(dfa)
    return name_subjs, dfa

In [ ]:
rbd_root = '.../RBD_root/'
ctrl_root = '.../Control_root/'
path_out = '.../Results/'

In [ ]:
# Set frequency limits
f_min = 2
f_max = 90
scale_freq='log'

freq_vals, _ = create_frequency_axis(f_max=f_max, f_min=f_min, scale_freq=scale_freq)
freq_vals = freq_vals[:30]

In [ ]:
name_subj_r1, dfa_r1 = extract_dfa_new(sorted(glob.glob(os.path.join(rbd_root + 'dfa/run-01/','sub*'))))
_, dfa_C = extract_dfa_new(sorted(glob.glob(os.path.join(rbd_root + 'dfa/ctrl/','sub*'))))

_, fei_r1 = extract_fei_new(sorted(glob.glob(os.path.join(rbd_root + 'fei/run-01/','*fei06*'))))
_, fei_C = extract_fei_new(sorted(glob.glob(os.path.join(rbd_root + 'fei/ctrl/','*fei06*'))))

_, bis_r1 = extract_bis_new(sorted(glob.glob(os.path.join(rbd_root + 'bis/run-01/','sub*'))))
_, bis_C = extract_bis_new(sorted(glob.glob(os.path.join(rbd_root + 'bis/ctrl/','sub*'))))

#### Compute confidence interval (bootstrap)

In [ ]:
perc_dfa_r1 = compute_bootstraped_percentile(dfa_r1[:,:30],1000)
perc_dfa_C = compute_bootstraped_percentile(dfa_C[:,:30],1000)

perc_bis_r1 = compute_bootstraped_percentile(bis_r1[:,:30],1000)
perc_bis_C = compute_bootstraped_percentile(bis_C[:,:30],1000)

perc_fei_r1 = compute_bootstraped_percentile(fei_r1[:,:30],1000)
perc_fei_C = compute_bootstraped_percentile(fei_C[:,:30],1000)

#### Wilcoxon to test difference between groups

In [ ]:
freq_dfa_r1c, freq_dfa_r1c_corrected, pdfa, p_Corr_dfa = compute_pvalue(dfa_r1, dfa_C, freq_vals)

freq_bis_r1c, freq_bis_r1c_corrected, pbis, p_Corr_bis = compute_pvalue(bis_r1, bis_C, freq_vals)

freq_fei_r1c, freq_fei_r1c_corrected, pfei, p_Corr_fei = compute_pvalue(fei_r1, fei_C, freq_vals)

#### Figure 1

In [ ]:
#Panels 
fig = plt.figure(figsize=(17.6/2.54, 6/2.54), layout='constrained')

gs = fig.add_gridspec(nrows=3, ncols=3)

ax1 = fig.add_subplot(gs[:-1, 0])
ax2 = fig.add_subplot(gs[:-1, 1])
ax3 = fig.add_subplot(gs[:-1, 2])

ax4 = fig.add_subplot(gs[-1, 0])
ax5 = fig.add_subplot(gs[-1, 1])
ax6 = fig.add_subplot(gs[-1, 2])

a_size=6
l_size=8
m_size=6

# Axis 1
plot_curve(ax1, freq_vals, dfa_r1[:,:30], 'blue', '-', 'iRBD - baseline')
plot_curve(ax1, freq_vals, dfa_C[:,:30], 'grey', '-', 'HC')
plot_stats(ax1, freq_dfa_r1c, freq_dfa_r1c_corrected, .875)
ax1.set_ylim([.55, .9])
ax1.set_ylabel('DFA', fontsize=l_size)


# Axis 2
plot_curve(ax2, freq_vals, bis_r1[:,:30], 'blue', '-', 'iRBD - baseline')
plot_curve(ax2, freq_vals, bis_C[:,:30], 'grey', '-', 'HC')
plot_stats(ax2, freq_bis_r1c, freq_bis_r1c_corrected, 4.75)
ax2.set_ylim([2, 5])
ax2.set_ylabel('BiS', fontsize=l_size)

# Axis 3
plot_curve(ax3, freq_vals, fei_r1[:,:30], 'blue', '-', 'iRBD - baseline')
plot_curve(ax3, freq_vals, fei_C[:,:30], 'grey', '-', 'HC')
plot_stats(ax3, freq_fei_r1c, freq_fei_r1c_corrected, 1.075)
ax3.set_ylim([.7, 1.1])
ax3.set_ylabel('fEI', fontsize=l_size)

ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)

ax3.legend(loc='lower right', fontsize=a_size, frameon=False)

# Cohen's d
d_bis = list()
d_fei = list()
d_dfa = list()

for iF in range(len(freq_vals)):

    d_bis.append(cohend(bis_r1[:,iF], bis_C[:,iF], None))
    d_fei.append(cohend(fei_r1[:,iF], fei_C[:,iF], None))
    d_dfa.append(cohend(dfa_r1[:,iF], dfa_C[:,iF], None))
    
    
d_dfa = np.asarray(d_dfa)
d_bis = np.asarray(d_bis)
d_fei = np.asarray(d_fei)

# Axis 4
ax4.semilogx(freq_vals, d_dfa, color='black')
ax4.semilogx(freq_vals, d_dfa, color='black')
ax5.semilogx(freq_vals, d_bis, color='black')
ax5.semilogx(freq_vals, d_bis, color='black')
ax6.semilogx(freq_vals, d_fei, color='black')
ax6.semilogx(freq_vals, d_fei,color='black')

ax4.fill_between(freq_vals, 1.5, -1, where=(d_dfa > .5), alpha=0.2, color='black')
ax5.fill_between(freq_vals, 1.5, -1, where=(d_bis > .5), alpha=0.2, color='black')
ax6.fill_between(freq_vals, 1.5, -1, where=(d_fei > .5), alpha=0.2, color='black')

ax4.spines['right'].set_visible(False)
ax4.spines['top'].set_visible(False)
ax5.spines['right'].set_visible(False)
ax5.spines['top'].set_visible(False)
ax6.spines['right'].set_visible(False)
ax6.spines['top'].set_visible(False)

ax4.set_yticks([-1, 0, 1])
ax5.set_yticks([-1, 0, 1])
ax6.set_yticks([-1, 0, 1])

ax1.set_xticks([], [])
ax2.set_xticks([], [])
ax3.set_xticks([], [])
ax4.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70])
ax5.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70])
ax6.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70])

ax4.tick_params(labelsize=a_size)
ax5.tick_params(labelsize=a_size)
ax6.tick_params(labelsize=a_size)

ax4.set_xlabel('Frequencies [Hz]', fontsize=l_size)
ax5.set_xlabel('Frequencies [Hz]', fontsize=l_size)
ax6.set_xlabel('Frequencies [Hz]', fontsize=l_size)
ax4.set_ylabel("Cohen's d", fontsize=l_size)

fig.align_ylabels([(ax1,ax2,ax3),(ax4,ax5,ax5)])
plt.savefig(path_out + 'Crit_F2.svg', dpi=600, bbox_inches = "tight")
plt.savefig(path_out + 'Crit_F2.png', dpi=600, bbox_inches = "tight")

In [ ]:
import pandas as pd 
df_dfa = pd.DataFrame()
df_dfa['Frequencies [Hz]'] = freq_vals
df_dfa['$\it{p}$'] = pdfa
df_dfa['$\it{p}$ corrected'] =  p_Corr_dfa

df_bis = pd.DataFrame()
df_bis['Frequencies [Hz]'] = freq_vals
df_bis['$\it{p}$'] = pbis
df_bis['$\it{p}$ corrected'] =  p_Corr_bis

df_fei = pd.DataFrame()
df_fei['Frequencies [Hz]'] = freq_vals
df_fei['$\it{p}$'] = pfei
df_fei['$\it{p}$ corrected'] = p_Corr_fei

with pd.ExcelWriter('SupplementaryTables_F3.xlsx') as writer:
    df_dfa.set_index('Frequencies [Hz]').to_excel(writer, sheet_name='Figure3A')
    df_bis.set_index('Frequencies [Hz]').to_excel(writer, sheet_name='Figure3C')
    df_fei.set_index('Frequencies [Hz]').to_excel(writer, sheet_name='Figure3D')